In [2]:
#本案例中利用前三个char测最后一个char
# LSTM讲解  https://zhuanlan.zhihu.com/p/32085405
# b =10   m=max_len=n_step=3     v=n_class=26    h=n_hidden=128

# input = (b,m)  ========>( np.eye(v)[input]) 操作  ) =====>(b,m,v)
# X.transpose(0, 1)         X :(b,m,v)===>(m,b,v)



#      X :(m,b,v)
#hidden_state =  [num_layers(=1) * num_directions(=1), b, h]=(1,b,h)
# cell_state =   [num_layers(=1) * num_directions(=1), b, h]=(1,b,h)


# self.lstm = nn.LSTM(input_size=v, hidden_size=h)   #(v,h)   
#LSTM 操作：    outputs, (_, _) = self.lstm(input, (hidden_state, cell_state))
                                #self.lstm(   (m,b,v) , ( (1,b,h),(1,b,h)  )  

 # 得到的outputs : [m, b, num_directions(=1) * h] = (m,b,h)=[3, 10, 128]


# 然后用outpu中的最后一层：outputs = outputs[-1]              # [b, num_directions(=1) * h]=(b,h)
    
 # (b, num_directions(=1) * h )  *    (num_directions(=1) * h ,v)= (b,h) * (h,v) = (b,v)
#(b,v)与实际的y做softmax得到loss 

In [3]:

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

dtype = torch.FloatTensor

char_arr = [c for c in 'abcdefghijklmnopqrstuvwxyz']
word_dict = {n: i for i, n in enumerate(char_arr)}
number_dict = {i: w for i, w in enumerate(char_arr)}
n_class = len(word_dict) # number of class(=number of vocab)

seq_data = ['make', 'need', 'coal', 'word', 'love', 'hate', 'live', 'home', 'hash', 'star']

print(' char_arr=',char_arr)
print(' word_dict=',word_dict)
print(' number_dict=',number_dict)
print("n_class = " ,n_class)

 char_arr= ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
 word_dict= {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25}
 number_dict= {0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h', 8: 'i', 9: 'j', 10: 'k', 11: 'l', 12: 'm', 13: 'n', 14: 'o', 15: 'p', 16: 'q', 17: 'r', 18: 's', 19: 't', 20: 'u', 21: 'v', 22: 'w', 23: 'x', 24: 'y', 25: 'z'}
n_class =  26


In [4]:
# TextLSTM Parameters
n_step = 3
n_hidden = 128

def make_batch(seq_data):
    input_batch, target_batch = [], []

    for seq in seq_data:
        input = [word_dict[n] for n in seq[:-1]] # 'm', 'a' , 'k' is input
        target = word_dict[seq[-1]] # 'e' is target
        input_batch.append(np.eye(n_class)[input])
        target_batch.append(target)

    return Variable(torch.Tensor(input_batch)), Variable(torch.LongTensor(target_batch))



In [5]:
class TextLSTM(nn.Module):
    def __init__(self):
        super(TextLSTM, self).__init__()
        #如果bi_lstm下面的参数改为 nn.LSTM(input_size=n_class, hidden_size=n_hidden, bidirectional=True)
        #如果bi_lstm             self.W = nn.Parameter(torch.randn([n_hidden * 2, n_class]).type(dtype))
        self.lstm = nn.LSTM(input_size=n_class, hidden_size=n_hidden)   #(v,h)
        self.W = nn.Parameter(torch.randn([n_hidden, n_class]).type(dtype))  #(h,v)
        self.b = nn.Parameter(torch.randn([n_class]).type(dtype))

    def forward(self, X):
        input = X.transpose(0, 1)  # X : [m, b, v]

        hidden_state = Variable(torch.zeros(1, len(X), n_hidden))   # [num_layers(=1) * num_directions(=1), b, h]=(1,b,h)
        cell_state = Variable(torch.zeros(1, len(X), n_hidden))     # [num_layers(=1) * num_directions(=1), b, h]=(1,b,h)

        outputs, (_, _) = self.lstm(input, (hidden_state, cell_state))  #    self.lstm(   (m,b,v) , ( (1,b,h),(1,b,h)  )  
        # print("#",outputs.size())      #(m,b,h)=[3, 10, 128]
        outputs = outputs[-1]  # [b,h]
        model = torch.mm(outputs, self.W) + self.b  # model : [b,v] =  (h,v) *(h,b)
        return model

input_batch, target_batch = make_batch(seq_data)
# print(input_batch.size(),target_batch.size())
#input_batch : [b,m, v]          # target_batch :[b]
model = TextLSTM()
print(model)
'''
TextLSTM(
  (lstm): LSTM(26, 128)
)
'''
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

output = model(input_batch)

# Training
for epoch in range(5000):
    optimizer.zero_grad()

    output = model(input_batch)   #(b,v)
    loss = criterion(output, target_batch)
    if (epoch + 1) % 1000 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

    loss.backward()
    optimizer.step()

TextLSTM(
  (lstm): LSTM(26, 128)
)


Epoch: 1000 cost = 0.000120


Epoch: 2000 cost = 0.000029


Epoch: 3000 cost = 0.000012


Epoch: 4000 cost = 0.000006


Epoch: 5000 cost = 0.000003


In [6]:
inputs = [sen[:3] for sen in seq_data]

predict = model(input_batch).data.max(1, keepdim=True)[1]
print(inputs, '->', [number_dict[n.item()] for n in predict.squeeze()])

['mak', 'nee', 'coa', 'wor', 'lov', 'hat', 'liv', 'hom', 'has', 'sta'] -> ['e', 'd', 'l', 'd', 'e', 'e', 'e', 'e', 'h', 'r']
